In [1]:
#from google.colab import drive
#drive.mount('/content/drive')


In [2]:
#%cd /content/drive/My\ Drive/pied-paper

In [3]:
!pip install --upgrade torch==1.8.0
!pip install --upgrade torchtext==0.9.0
# !pip show torch
# !pip show torchtext

You should consider upgrading via the 'C:\Users\paulh\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\paulh\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [10]:
!pip install sklearn

You should consider upgrading via the 'C:\Users\paulh\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [11]:
import numpy as np
import pandas as pd

news_path = 'data/news.csv'
destination_folder = 'preprocessed'

In [12]:
news_raw = pd.read_csv(news_path)
news_raw.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [13]:
first_n_words = 200

def trim_string(x):
    x = x.split(maxsplit=first_n_words)
    x = ' '.join(x[:first_n_words])
    return x

In [14]:
news_raw['label'] = (news_raw['label'] == 'FAKE').astype(int)
news_raw['titletext'] = news_raw['title'] + ' ' + news_raw['text']
news_raw = news_raw.reindex(columns=['label', 'title', 'text', 'titletext'])

news_raw.drop(news_raw[news_raw.text.str.len() < 5].index, inplace=True)

news_raw['text'] = news_raw['text'].apply(trim_string)
news_raw['titletext'] = news_raw['titletext'].apply(trim_string)

news_real = news_raw[news_raw['label'] == 0]
news_fake = news_raw[news_raw['label'] == 1]

In [15]:
from sklearn.model_selection import train_test_split

train_test_ratio = 0.10
train_valid_ratio = 0.80

# Train-test split
news_real_full_train, news_real_test = train_test_split(news_real, train_size = train_test_ratio, random_state = 1)
news_fake_full_train, news_fake_test = train_test_split(news_fake, train_size = train_test_ratio, random_state = 1)

# Train-valid split
news_real_train, news_real_valid = train_test_split(news_real_full_train, train_size = train_valid_ratio, random_state = 1)
news_fake_train, news_fake_valid = train_test_split(news_fake_full_train, train_size = train_valid_ratio, random_state = 1)

In [16]:
print(news_real_train.shape)
print(news_real_valid.shape)
print(news_real_test.shape)

(253, 4)
(64, 4)
(2854, 4)


In [17]:
# Concatenate splits of different labels
news_train = pd.concat([news_real_train, news_fake_train], ignore_index=False, sort=False)
news_valid = pd.concat([news_real_valid, news_fake_valid], ignore_index=False, sort=False)
news_test = pd.concat([news_real_test, news_fake_test], ignore_index=False, sort=False)

In [18]:
print(news_train.shape)
print(news_valid.shape)
print(news_test.shape)

(502, 4)
(127, 4)
(5670, 4)


In [19]:
# Write preprocessed data
news_train.to_csv(destination_folder + '/train.csv', index=False)
news_valid.to_csv(destination_folder + '/valid.csv', index=False)
news_test.to_csv(destination_folder + '/test.csv', index=False)